# [LVV-T1876] - Settling Time After a Slew

Notebook containing data analysis for the [LVV-T1876] test case.  
~The script used to run this test case can be found in [lsst-ts/ts_m1m3supporttesting/M13T012.py].~


[LVV-T1876]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/2237
[lsst-ts/ts_m1m3supporttesting/M13T012.py]: https://github.com/lsst-ts/ts_m1m3supporttesting/blob/develop/M13T012.py


see [LVV-T2732_analysis_mt_encoder_slew_jitter.ipynb] for jitter  ideas?

[LVV-T2732_analysis_mt_encoder_slew_jitter.ipynb]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/notebooks/tel_and_site/subsys_req_ver/tma/LVV-T2732_analysis_mt_encoder_slew_jitter.ipynb

Some pending items(July 8th 2023) --> add as issues:
1. Get some RMS requirements 
2. Tweak parameters for stationarity, for a robust measurement


### Prepare Notebook

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from astropy.time import Time, TimezoneInfo
from statsmodels.tsa.stattools import adfuller

from lsst.sitcom import vandv
from lsst.ts.idl.enums import MTM1M3

from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState
from lsst.summit.utils.efdUtils import getEfdData, makeEfdClient

In [ ]:
# create a client to retrieve datasets in the EFD database
client = makeEfdClient()

### Define functions

#### determine settling time

In [ ]:
def determineSettleTime(df_ims, #input data frame
                        reference_t = '2023-06-01T06:00:0Z', #time for slew stop
                        lo_delta_t = 5, #in seconds
                        hi_delta_t = 30, #in seconds
                        imsColumn = 'xPosition', #IMS column
                        rmsReq = 1e-2, #requirement in appropriate units
                        req_delta_t = 3): #time for settling, in seconds
    
    #T0 and T1 defines the window for requirement to be met
    T0 = pd.to_datetime(reference_t)
    T1 = T0 + pd.to_timedelta(req_delta_t,unit='s')
    # delta_window is the time window for plot and reference for mean, before and after T0
    delta_window = [pd.Timedelta(lo_delta_t, "seconds"),pd.Timedelta(hi_delta_t, "seconds")] 

    if 'Position' in imsColumn:
        units = 'mm'
    else:
        units = 'deg'

    #zoom around the T0 of interest
    TZoom = [T0-delta_window[0],  
            T0+delta_window[1]] 
    
    targetVariablePlot = df_ims[imsColumn][TZoom[0]:TZoom[1]] #complete plot range
    targetVariable = df_ims[imsColumn][T0:T1] #from slew stop to end of window
    idxT0 = df_ims.index[df_ims.index.get_indexer([pd.to_datetime(T0)], method='nearest')]
    targetVariableReference = [df_ims[imsColumn][idxT0],df_ims[imsColumn][-1]] #at slew stop and end of plot
    
    if len(targetVariablePlot.index) == 0:
        print('Data frame is empty')
        return -1
    
    rms = np.std(targetVariable-targetVariableReference[1]) #rms of difference wrt to end of plot, in window

    title = imsColumn
    label = imsColumn + '(' + units +') wrt T0 (slew stop)'

    fig = plt.figure()
    plt.plot(targetVariablePlot-targetVariableReference[0][0],color='red', lw="0.5", label=label)
    plt.axvline(T0, lw="2", c="k", label='Slew stop')
    #plt.axhline(-req)
    #plt.axhline(req)
    plt.xlabel("Time [UTC]")
    plt.ylabel(label)
    plt.text(0.55, 0.90, 'RMS =' + ' {:.2e} '.format(rms) + units + 
             ' \nin 3s window, wrt end of plot', 
             transform=fig.transFigure)
    fig.autofmt_xdate()
    fig.tight_layout()

    ## recomputing RMS for the whole range since T0
    targetVariable = df_ims[imsColumn][T0:TZoom[1]]
    rms = (targetVariable - targetVariableReference[1]).rolling(20).std()
    plt.plot(rms, label='RMS wrt value at the end of plot, from T0')
    plt.axvspan(T0, T1, alpha=0.5, color='green', label='3s tolerance window')
    plt.legend(loc="upper right")
    #print(rms)

    ## compute time for settling
    fig = plt.figure()    
    rms = rms.dropna() # remove NaNs created by the rolling function
    lapse = 100 # to be adjusted
    pvalue = 0.05 ## if lower, less tolerant to finding stationarity, to be adjusted
    k= 0
    for i in range(0,len(rms)-lapse):
        adf = adfuller(rms[i:i+lapse], autolag='AIC')
        if adf[1] < pvalue: # is non-stationary if null hypothesis is true (large p-value)
            print(i, i+lapse, adf[1], pvalue)
            k = i+lapse
            print(rms.index[i], rms.index[k])
            break

    plt.plot(rms)
    plt.xlabel("Time [UTC]")
    plt.ylabel('RMS wrt value at the end of plot (mm)')    
    fig.autofmt_xdate()
    fig.tight_layout()
    plt.text(0.50, 0.93, 'Slew stop =' + t1.strftime('%H:%M:%S.%f'), transform=fig.transFigure)        
    plt.text(0.50, 0.90, 'Settling time (ADF test) =' + rms.index[k].strftime('%H:%M:%S.%f'), transform=fig.transFigure)        

    return rms.index[k]
    

#### compare IMS vs mount data

In [ ]:
def compareIMSMount(df_ims,
                   df_mtmount_ele,
                   df_mtmount_azi,
                   reference_t = '2023-06-01 06:53:37.326',
                   lo_delta_t = 0, #in seconds
                   hi_delta_t = 30): #in seconds
    
    fig,axs = plt.subplots(5,1, dpi=125, figsize=(6,8))
    t_start_plot = pd.to_datetime(reference_t) - pd.to_timedelta(lo_delta_t,unit='s') ##correcting by hand for TAI, is this correct?
    t_end_plot = pd.to_datetime(reference_t) + pd.to_timedelta(hi_delta_t,unit='s')
    print('reference_t:',reference_t)
    print('t_start:',t_start_plot)
    print('t_end:',t_end_plot)

    ax = axs[0]
    actpos_ele = df_mtmount_ele['actualPosition'][t_start_plot:t_end_plot]
    ax.plot(actpos_ele,color='red', lw="0.5")
    ax.axvline(pd.to_datetime(reference_t), lw="0.5", c="k")
    ax.set_ylabel("Elevation \nPosition\n[deg]")
    
    ax = axs[1]
    actpos_azi = df_mtmount_azi['actualPosition'][t_start_plot:t_end_plot]
    ax.plot(actpos_azi,color='red', lw="0.5")
    ax.axvline(pd.to_datetime(reference_t), lw="0.5", c="k")
    ax.set_ylabel("Azimuth \nPosition\n[deg]")

    t_start_plot = pd.to_datetime(reference_t) - pd.to_timedelta(lo_delta_t,unit='s') 
    t_end_plot = pd.to_datetime(reference_t) + pd.to_timedelta(hi_delta_t,unit='s')
    
    plotstring = ['xPosition','yPosition','zPosition']
    plotrange = np.arange(len(plotstring))    
    for i in plotrange:
        ax = axs[i+2]
        pos = df_ims[plotstring[i]][t_start_plot:t_end_plot]
        ax.plot(pos,color='red', lw="0.5")
        ax.axvline(pd.to_datetime(reference_t), lw="0.5", c="k")
        ax.set_ylabel(plotstring[i] + ' \n[mm]')
    ax.set_xlabel('UTC')
    fig.autofmt_xdate()
    fig.subplots_adjust(hspace=1)
    ### TBD: use a delta time wrt slew stop in x-label

In [ ]:
print(type(df_mtmount_azi))

### Get slew stops

In [ ]:
# Select data from a given date
dayObs = 20230623
eventMaker = TMAEventMaker()
events = eventMaker.getEvents(dayObs)

# Get lists of slew and track events
slews = [e for e in events if e.type==TMAState.SLEWING]
tracks = [e for e in events if e.type==TMAState.TRACKING]
print(f'Found {len(slews)} slews and {len(tracks)} tracks')

### Load data

In [ ]:
all_columns = ["xPosition", "xRotation", "yPosition", "yRotation", "zPosition", "zRotation"]
pos_columns = [c for c in all_columns if "Position" in c]
rot_columns = [c for c in all_columns if "Rotation" in c]

#### Using eventMaker 

In [ ]:
## Some settings
# define specific slew
i_slew = 30
# define padding after end of slew
postPadding = 30 # in seconds

In [ ]:
t0 = Time(slews[i_slew].begin, format='isot', scale='utc')
t0 = pd.to_datetime(t0.value, utc=True) # astropy Time to Timestamp conversion
t1 = Time(slews[i_slew].end, format='isot', scale='utc')
t1 = pd.to_datetime(t1.value, utc=True) # astropy Time to Timestamp conversion
print('Slew stop at:',t1)

# Get IMS data
# Note that we need data beyond the associated to the slew stop, 
# which just covers the slew itself, ie, while the telescope is moving (TBC)
df_ims = getEfdData(client, 'lsst.sal.MTM1M3.imsData', 
                    event=slews[i_slew], 
                    postPadding = postPadding)
df_ims = df_ims[all_columns]
# Convert meter to milimeter 
df_ims[pos_columns] = df_ims[pos_columns] * 1e3


In [ ]:
# Get mount data
df_mtmount_ele = getEfdData(client,'lsst.sal.MTMount.elevation', 
                            event=slews[i_slew], 
                            postPadding = postPadding)
df_mtmount_azi = getEfdData(client,'lsst.sal.MTMount.azimuth', 
                            event=slews[i_slew], 
                            postPadding = postPadding)

### Look at data

In [ ]:
%matplotlib inline
compareIMSMount(df_ims,df_mtmount_ele,df_mtmount_azi,
                t1,10,pd.to_timedelta(postPadding, unit="s"))

### Run test

In [ ]:
req_delta_t = 3 ## seconds
req_rms = 1e-2 

In [ ]:
#%%capture --no-display 
determineSettleTime(df_ims = df_ims, 
                    reference_t = t1, 
                    lo_delta_t = 5,
                    hi_delta_t = postPadding,
                    imsColumn = 'xPosition', 
                    rmsReq = req_rms, 
                    req_delta_t = delta_t)